In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import os
import os.path
from skimage.io import imread

def read_from_folder(folder):
    data_dir = os.path.join(os.getcwd(),folder)
    img_list = []
    for name in os.listdir(data_dir):
        img = imread(os.path.join(data_dir, name))
        img_list.append(img)
    return img_list

In [ ]:
imgs_per_cat = dict()
imgs_per_cat['papel'] = read_from_folder('papel')
imgs_per_cat['pedra'] = read_from_folder('pedra')
imgs_per_cat['tesoura'] = read_from_folder('tesoura')

In [ ]:
def show_images(imgs):
    C = 4
    N = len(imgs)
    R = (N - 1)//C + 1
    
    plt.figure(figsize=(C*3, R*3))
    for k, img in enumerate(imgs):
        plt.subplot(R, C, k+1)
        plt.imshow(img, cmap='gray')
    plt.show()

for cat, imgs in imgs_per_cat.items():
    print(f'{cat}: {len(imgs)} images')
    show_images(imgs)

In [ ]:
from skimage.measure import regionprops, perimeter

X = []
y = []

#Pegando caracteristicas específicas de cada imagem

for cat, imgs in imgs_per_cat.items():
    for img in imgs:
        props = regionprops(img)[0]
        eccentricity = props['eccentricity']
        hu_moments = props['moments_hu']
        area = props['area']
        perimeter = props['perimeter']
        if perimeter == 0.0:
            perimeter = 1.0
        compactness = 4.0*np.pi*area/perimeter
        feats = [eccentricity, compactness] + list(hu_moments)
        X.append(feats)
        y.append(cat)

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=42)

clf = RandomForestClassifier(n_estimators=100)

clf.fit(X_train, y_train)
y_pred = clf.predict(X_test)

accuracy_score(y_test, y_pred)

In [ ]:
import joblib

joblib.dump(clf, 'meu_modelo.joblib')

In [ ]:
# Lendo o modelo.
#clf = joblib.load('meu_modelo.joblib')

In [ ]:
#clf.predict([X_test[0]])